In [2]:
!pip install pymorphy2

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 912.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 51.4 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=ae3ab34f1b61d07bccde47cdd8837caa835ceacecdad895ab1e6ae87bee4177e
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [7]:
import csv
import pymorphy2

# Инициализация pymorphy2 для морфологического анализа
morph = pymorphy2.MorphAnalyzer()


class SearchEngineCSV:
    def __init__(self, csv_file):
        # Чтение и загрузка данных из CSV
        self.sentences = []
        self.tokens = []
        self.lemmas = []
        self.pos_tags = []

        with open(csv_file, newline='', encoding='utf-8') as file:
            reader = csv.reader(file)
            next(reader)  # Пропуск заголовка
            for row in reader:
                self.sentences.append(row[0])  # Предложения
                self.tokens.append(row[1].split(';'))  # Токены
                self.lemmas.append(row[2].split(';'))  # Леммы
                self.pos_tags.append(row[3].split(';'))  # Части речи

    def match_token(self, token, word, lemma, pos):
        """Проверка совпадения токена с запросом."""
        if '"' in token:
            return token.strip('"') == word

        if '+' in token:
            query_word, query_pos = token.split('+')
            return lemma == query_word and pos == query_pos

        if token.isupper():
            return pos == token

        return lemma == morph.parse(token)[0].normal_form

    def match_sequence(self, query_tokens, words, lemmas, pos_tags):
        """Проверка совпадения последовательности токенов."""
        for j, query_token in enumerate(query_tokens):
            if query_token.isupper():
                # Если запрос — это POS-тег, проверяем только POS-тег
                if pos_tags[j] != query_token:
                    return False
            else:
                # Иначе обрабатываем стандартные правила совпадения
                if not self.match_token(query_token, words[j], lemmas[j], pos_tags[j]):
                    return False
        return True

    def search(self, query):
        query_tokens = query.split()

        matches = []
        for idx, sentence in enumerate(self.sentences):
            words = self.tokens[idx]
            lemmas = self.lemmas[idx]
            pos_tags = self.pos_tags[idx]

            # Поиск последовательности токенов в предложении
            for i in range(len(words) - len(query_tokens) + 1):
                if len(words) != len(lemmas) or len(words) != len(pos_tags):
                    continue
                # Проверка последовательности для n-граммы
                if self.match_sequence(query_tokens, words[i:i+len(query_tokens)], lemmas[i:i+len(query_tokens)], pos_tags[i:i+len(query_tokens)]):
                    matches.append(sentence)
                    break

        return matches

csv_file = '/content/corpus_data_with_labels.csv'  # Путь к файлу
search_engine = SearchEngineCSV(csv_file)

# Код для выдачи запросов

print('Введите запрос')
for_search = input()
print(f'Найдено {len(search_engine.search(for_search))} вхождений')
print(search_engine.search(for_search))


Введите запрос
"меня"
Найдено 191 вхождений
['Бо́льшая часть из них, к счастию для вас, потеряна, а чемодан с остальными вещами, к счастию для меня, остался цел.', 'Это обстоятельство меня удивило.', 'Он лукаво улыбнулся и значительно взглянул на меня: — Вы верно недавно на Кавказе?', 'Уж я их знаю, меня не проведут!', 'Осетины шумно обступили меня и требовали на водку; но штабс-капитан так грозно на них прикрикнул, что они вмиг разбежались.', '— сказал я моему собеседнику: — у меня есть белый из Тифлиса; теперь холодно.', 'Он явился ко мне в полной форме и объявил, что ему велено остаться у меня в крепости.', 'Да, пожалуйста, зовите меня просто Максим Максимыч, и пожалуйста — к чему эта полная форма?', 'У меня было свое на уме.', 'Печорин встал, поклонился ей, приложив руку ко лбу и сердцу, и просил меня отвечать ей; я хорошо знаю по-ихнему, и перевел его ответ.', 'Мне вздумалось завернуть под навес, где стояли наши лошади, посмотреть, есть ли у них корм, и притом осторожность никогда

In [ ]:
# Примеры запросов
print(search_engine.search('сказать'))  # Любая форма слова "сказать"
print(search_engine.search('"меня"'))  # Точная форма "меня"
print(search_engine.search('человек+NOUN'))  # Лемма "человек" с POS-тегом "S"
print(search_engine.search('NOUN ADV VERB'))  # Существительное, глагол, наречие в последовательности